<a href="https://colab.research.google.com/github/swapnilmn/Assignment_03/blob/main/Question_4_Ass_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Question 4 (10 Marks)

You will now apply your best model on the test data (You shouldn't have used test data so far. All the above experiments should have been done using train and val data only). 

In [ ]:
import csv
import numpy as np
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import time
import matplotlib.pyplot as plt
import math
!pip install wandb -qU
import wandb

# Check if CUDA is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def my_function():
    # Install and import the wandb library for experiment tracking
    !pip install wandb -qU
    import wandb
    
    # Login to wandb with your API key
    !wandb login 5dbfdc4a0de265f2dd69d623f169b7884aa4436f
    
    # Mount Google Drive to access files
    from google.colab import drive
    drive.mount('/content/drive')

# Call the function
my_function()


In [ ]:
import math
import time
import torch
import numpy as np

class Helper_Functions:
    @staticmethod
    def Time(s):
        # Converts seconds to minutes and seconds
        m = math.floor(s / 60)
        s -= m * 60
        return '%dm %ds' % (m, s)

    @staticmethod
    def Span(since, percent):
        # Calculates the elapsed time and estimated remaining time
        now = time.time()
        s = now - since
        es = s / percent
        rs = es - s
        return '%s (- %s)' % (Helper_Functions.Time(s), Helper_Functions.Time(rs))

    @staticmethod
    def key_printing(d, v):
        # Prints keys from a dictionary that have specific values
        f = [k for k, val in d.items() if val in v]
        if f:
            s = ''.join(f)
            if s[-1] == '\n':
                s = s[:-1]
            elif s[0] == '\t':
                s = s[1:]
        else:
            print("No keys")

        return s

    @staticmethod
    def char_acc(t, o):
        # Calculates character-level accuracy between predicted and target tensors
        with torch.no_grad():
            se = [[o[i][j].item() == t[i][j].item() for j in range(t.shape[1])] for i in range(t.shape[0])]
            c = np.sum(se)
            tc = np.sum([len(row) for row in se])
        return c / tc

    @staticmethod
    def word_acc(t, o):
        # Calculates word-level accuracy between predicted and target tensors
        o1 = torch.argmax(o, dim=1)
        with torch.no_grad():
            c = sum([(o1[i] == t[i]).sum().item() == t.shape[1] for i in range(t.shape[0])])
        return c / t.shape[0]

    @staticmethod
    def sample_equidi_pts(d, e):
        # Samples equidistant points from a given list
        s = len(d) // e
        i = np.arange(0, len(d), s)
        p = [d[j] for j in i]
        return p


HF = Helper_Functions()

# Creating a dictionary of helper functions
variables = {
    'asMinutes': HF.Time,
    'timeSince': HF.Span,
    'print_keys_for_values': HF.key_printing,
    'char_level_accuracy': HF.char_acc,
    'word_level_accuracy': HF.word_acc,
    'sample_equidistant_points': HF.sample_equidi_pts
}

# Accessing the variables
asMinutes = variables['asMinutes']
timeSince = variables['timeSince']
print_keys_for_values = variables['print_keys_for_values']
char_level_accuracy = variables['char_level_accuracy']
word_level_accuracy = variables['word_level_accuracy']
sample_equidistant_points = variables['sample_equidistant_points']

# Usage
print(asMinutes)  # Prints the formatted time in minutes and seconds
print(timeSince)  # Prints the elapsed time and estimated remaining time
print(print_keys_for_values)  # Prints keys from a dictionary that have specific values
print(char_level_accuracy)  # Calculates and returns character-level accuracy
print(word_level_accuracy)  # Calculates and returns word-level accuracy
print(sample_equidistant_points)  # Samples equidistant points from a given list


In [ ]:
# Define a dictionary of variables and their corresponding characters
variables = {
    'start_char': '\t',     # Represents the tab character
    'end_char': '\n',       # Represents the newline character
    'blank_char': ' ',      # Represents a blank space character
    'unknown_char': '\r'    # Represents a carriage return character
}

# Assign the value of 'start_char' from the 'variables' dictionary to the variable 'start_char'
start_char = variables['start_char']

# Assign the value of 'end_char' from the 'variables' dictionary to the variable 'end_char'
end_char = variables['end_char']

# Assign the value of 'blank_char' from the 'variables' dictionary to the variable 'blank_char'
blank_char = variables['blank_char']

# Assign the value of 'unknown_char' from the 'variables' dictionary to the variable 'unknown_char'
unknown_char = variables['unknown_char']


In [ ]:
import csv  # Import the csv module
import torch  # Import the torch module

class DataPreProcessor:
    def __init__(self):
        self.file_paths = {
            'file1': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_test.csv',
            'file2': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv',
            'file3': '/content/drive/MyDrive/aksharantar_sampled/mar/mar_train.csv'
        }

        self.variables = {
            'start_char': '\t',      # Represents the tab character
            'end_char': '\n',        # Represents the newline character
            'blank_char': ' ',       # Represents a blank space character
            'unknown_char': '\r'     # Represents a carriage return character
        }

    def read_csv_files(self):
        fr = {}  # Dictionary to store csv reader objects
        hd = {}  # Dictionary to store headers of each file
        dt = {}  # Dictionary to store data from each file

        for key, fp in self.file_paths.items():
            file = open(fp)  # Open the file with the given file path
            csv_reader = csv.reader(file)  # Create a csv reader object
            fr[key] = csv_reader  # Store the csv reader object in the dictionary
            hd[key] = next(csv_reader)  # Retrieve the headers from the csv reader
            dt[key] = []  # Initialize an empty list to store the data

            for row in csv_reader:
                dt[key].append(row)  # Append each row of data to the corresponding list

            file.close()  # Close the file after reading

        # Return the headers and data from each file
        return hd['file1'], hd['file2'], hd['file3'], dt['file1'], dt['file2'], dt['file3']

    def Reading_Data(self, lst):
        i = [pair[0] for pair in lst]  # Extract the first element from each pair in the given list
        t = [pair[1] for pair in lst]  # Extract the second element from each pair in the given list
        return i, t  # Return the extracted first elements as 'i' and second elements as 't'


    def Dict_lang(self, inputs, targets):
        i_dict = {}         # Dictionary to store character-to-index mapping for inputs
        max_i_length = 0    # Variable to track maximum input string length
        i_char = []         # List to store unique characters in inputs

        t_dict = {}         # Dictionary to store character-to-index mapping for targets
        max_t_length = 0    # Variable to track maximum target string length
        t_char = []         # List to store unique characters in targets

        # Encoding Inputs and updating i_dict
        for string in inputs:
            max_i_length = max(len(string), max_i_length)  # Update maximum input string length
            for char in string:
                if char not in i_dict:
                    i_dict[char] = len(i_char)   # Assign a unique index to each unique character
                    i_char.append(char)         # Store the unique character in the list

        if self.variables['blank_char'] not in i_dict:
            i_dict[self.variables['blank_char']] = len(i_char)   # Assign index to the blank character if not present
            i_char.append(self.variables['blank_char'])

        i_dict[self.variables['unknown_char']] = len(i_char)     # Assign index to the unknown character
        i_char.append(self.variables['unknown_char'])

        if self.variables['start_char'] not in t_dict:
            t_dict[self.variables['start_char']] = len(t_char)   # Assign index to the start character if not present
            t_char.append(self.variables['start_char'])

        # Encoding Targets and updating t_dict
        for string in targets:
            max_t_length = max(len(string) + 2, max_t_length)    # Update maximum target string length
            for char in string:
                if char not in t_dict:
                    t_dict[char] = len(t_char)   # Assign a unique index to each unique character
                    t_char.append(char)         # Store the unique character in the list

        if self.variables['end_char'] not in t_dict:
            t_dict[self.variables['end_char']] = len(t_char)     # Assign index to the end character if not present
            t_char.append(self.variables['end_char'])

        if self.variables['blank_char'] not in t_dict:
            t_dict[self.variables['blank_char']] = len(t_char)   # Assign index to the blank character if not present
            t_char.append(self.variables['blank_char'])

        return i_dict, max_i_length, i_char, t_dict, max_t_length, t_char

    def encoding_words(self, wl, ld, ml, l):
        encds = []   # List to store the encoded sequences
        for w in wl:
            encd = [ld[c] if c in ld else ld[self.variables['unknown_char']] for c in w]
            # Assign the index of each character in the word, or the index of unknown character if not present

            if l == 0:
                encd.extend([ld[self.variables['blank_char']]] * (ml - len(encd)))
                # If 'l' is 0 (inputs), pad the sequence with blank character indices up to the maximum length
            if l == 1:
                encd = [ld[self.variables['start_char']]] + encd + [ld[self.variables['end_char']]]
                encd.extend([ld[self.variables['blank_char']]] * (ml - len(encd)))
                # If 'l' is 1 (targets), add start and end character indices to the sequence and pad it with blank character indices up to the maximum length

            encds.append(encd)  # Append the encoded sequence to the list

        return encds  # Return the list of encoded sequences


    def Tokenzieations(self, train, val, test, input_dict, target_dict, max_input_length, max_target_length):
        t_i, t_t = self.Reading_Data(train)  # Reading train data
        te_i, te_t = self.Reading_Data(test)  # Reading test data
        v_i, v_t = self.Reading_Data(val)  # Reading validation data

        e_t_i = self.encoding_words(t_i, input_dict, max_input_length, 0)  # Encoding train inputs
        e_t_t = self.encoding_words(t_t, target_dict, max_target_length, 1)  # Encoding train targets
        e_v_i = self.encoding_words(v_i, input_dict, max_input_length, 0)  # Encoding validation inputs
        e_v_t = self.encoding_words(v_t, target_dict, max_target_length, 1)  # Encoding validation targets
        e_te_i = self.encoding_words(te_i, input_dict, max_input_length, 0)  # Encoding test inputs
        e_te_t = self.encoding_words(te_t, target_dict, max_target_length, 1)  # Encoding test targets

        return {
            'encoded_train_inputs': e_t_i,
            'encoded_train_targets': e_t_t,
            'encoded_val_inputs': e_v_i,
            'encoded_val_targets': e_v_t,
            'encoded_test_inputs': e_te_i,
            'encoded_test_targets': e_te_t
        }

    def tensor_pair_conversion(self, train_inputs, train_targets):
        pairs = [(torch.tensor(input_data), torch.tensor(target_data)) for input_data, target_data in
                zip(train_inputs, train_targets)]  # Creating pairs of tensor inputs and targets
        return pairs

DPP = DataPreProcessor()

# Dictionary of functions
function_dict = {
    'tensor_pair_conversion': DPP.tensor_pair_conversion,
    'Tokenzieations': DPP.Tokenzieations,
    'encoding_words': DPP.encoding_words,
    'Reading_Data': DPP.Reading_Data,
    'Dict_lang': DPP.Dict_lang,
    'read_csv_files': DPP.read_csv_files
}

tensor_pair_conversion = function_dict['tensor_pair_conversion']
Tokenzieations = function_dict['Tokenzieations']
encoding_words = function_dict['encoding_words']
Reading_Data = function_dict['Reading_Data']
Dict_lang = function_dict['Dict_lang']
read_csv_files = function_dict['read_csv_files']

# Call the function and assign the returned values to variables
h1, h2, h3, test, val, train = read_csv_files()

train_inputs, train_targets = Reading_Data(train)  # Reading train data
test_inputs, test_targets = Reading_Data(test)  # Reading test data
val_inputs, val_targets = Reading_Data(val)  # Reading validation data

# print(train_inputs[1])  # Print train input at index 1
# print(train_targets[1])  # Print train target at index 1

input_dict, max_input_length, input_char, target_dict, max_target_length, target_char = Dict_lang(train_inputs + val_inputs + test_inputs, train_targets + val_targets + test_targets)  # Generating dictionaries for inputs and targets

result = Tokenzieations(train, val, test, input_dict, target_dict, max_input_length, max_target_length)  # Tokenizing the data

encoded_train_inputs = result['encoded_train_inputs']  # Encoded train inputs
encoded_train_targets = result['encoded_train_targets']  # Encoded train targets
encoded_val_inputs = result['encoded_val_inputs']  # Encoded validation inputs
encoded_val_targets = result['encoded_val_targets']  # Encoded validation targets
encoded_test_inputs = result['encoded_test_inputs']  # Encoded test inputs
encoded_test_targets = result['encoded_test_targets']  # Encoded test targets

r = random.randint(0,100)  # Generate a random number between 0 and 100
# print(print_keys_for_values(input_dict, encoded_train_inputs[int(r)]))  # Print the keys corresponding to the values in input_dict for the randomly selected encoded train input
# print(print_keys_for_values(target_dict, encoded_train_targets[int(r)]))  # Print the keys corresponding to the values in target_dict for the randomly selected encoded train target

encoded_train_pairs = tensor_pair_conversion(encoded_train_inputs, encoded_train_targets)  # Convert encoded train inputs and targets into pairs of tensors
encoded_val_pairs = tensor_pair_conversion(encoded_val_inputs, encoded_val_targets)  # Convert encoded validation inputs and targets into pairs of tensors
encoded_test_pairs = tensor_pair_conversion(encoded_test_inputs, encoded_test_targets)  # Convert encoded test inputs and targets into pairs of tensors

pairs = (encoded_train_pairs, encoded_val_pairs, encoded_test_pairs)  # Combine pairs into a tuple
pair = random.choice(encoded_train_pairs)  # Randomly select a pair from the encoded train pairs

# print(print_keys_for_values(input_dict, pair[0]))  # Print the keys corresponding to the values in input_dict for the selected pair input
# print(print_keys_for_values(target_dict, pair[1]))  # Print the keys corresponding to the values in target_dict for the selected pair target


In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, device, cell_type, vocab_size, embed_dim, hidden_size, num_layers=1, bidirectional=False, dropout_p=0):
        super(EncoderRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)  # Embedding layer for converting input indices to dense vectors
        self.hidden_size = hidden_size  # Size of the hidden state in the RNN
        self.num_layers = num_layers  # Number of layers in the RNN
        self.bidirectional = bidirectional  # Flag indicating whether the RNN is bidirectional or not
        self.cell_type = cell_type  # Type of RNN cell ('lstm', 'rnn', or 'gru')
        self.dropout_p = dropout_p  # Dropout probability
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer for regularization
        
        # Initialize the RNN cell based on the specified cell type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(embed_dim, hidden_size, num_layers=self.num_layers, batch_first=True, dropout=dropout_p, bidirectional=self.bidirectional)

    def forward(self, x, hidden, cell):
        out = self.embedding(x)  # Perform embedding lookup to get dense representations of input indices
        out = self.dropout(out)  # Apply dropout to the input embeddings
        if self.cell_type == 'lstm':
            out, (hidden, cell) = self.rnn(out, (hidden, cell))  # Forward pass through the LSTM
            return out, hidden, cell
        elif self.cell_type == 'rnn':
            out, hidden = self.rnn(out, hidden)  # Forward pass through the RNN
            return out, hidden
        elif self.cell_type == 'gru':
            out, hidden = self.rnn(out, hidden)  # Forward pass through the GRU
            return out, hidden
    
    def init_hidden(self, batch_size):
        # Initialize the hidden and cell states with random values
        hidden = torch.randn((1 + int(self.bidirectional)) * self.num_layers, batch_size, self.hidden_size, device=device)
        cell = torch.randn((1 + int(self.bidirectional)) * self.num_layers, batch_size, self.hidden_size, device=device)
        return hidden, cell


class DecoderRNN(nn.Module):
    def __init__(self, device, cell_type, output_vocab, embed_size, hidden_size, max_length, dropout_p=0.1, num_layers=1, bidirectional=False):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size  # Hidden size of the decoder
        self.output_size = output_vocab  # Size of the output vocabulary
        self.embed_size = embed_size  # Size of the input embedding
        self.dropout_p = dropout_p  # Dropout probability
        self.cell_type = cell_type  # Type of RNN cell (lstm, gru, rnn)
        self.max_length = max_length  # Maximum length of input sequence
        self.device = device  # Device (e.g., CPU or GPU) to be used for computations
        self.num_layers = num_layers  # Number of layers in the decoder
        self.embedding_decoder = nn.Embedding(self.output_size, self.embed_size)  # Embedding layer for the decoder
        self.dropout = nn.Dropout(self.dropout_p)  # Dropout layer
        self.bidirectional = bidirectional  # Flag indicating if the encoder is bidirectional

        # Determine the type of RNN cell based on the given cell_type
        if cell_type == 'lstm':
            self.rnn = nn.LSTM(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'gru':
            self.rnn = nn.GRU(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)
        elif cell_type == 'rnn':
            self.rnn = nn.RNN(self.embed_size, hidden_size, num_layers=self.num_layers, batch_first=True, bidirectional=self.bidirectional, dropout=self.dropout_p)

        self.out = nn.Linear((1 + int(self.bidirectional)) * self.hidden_size, self.output_size)  # Linear layer for output prediction
        self.out_activation = nn.LogSoftmax(dim=-1)  # Log softmax activation for output probabilities

    def forward(self, input, hidden, cell):
        input = input.unsqueeze(1)  # Add a singleton dimension to the input tensor
        embedded_decoder = self.embedding_decoder(input)  # Apply embedding to the input
        embedded_decoder = self.dropout(embedded_decoder)  # Apply dropout to the embedded input

        # Pass the embedded input through the RNN cell
        if self.cell_type == 'lstm':
            output, (hidden, cell) = self.rnn(embedded_decoder, (hidden, cell))
        elif self.cell_type == 'gru':
            output, hidden = self.rnn(embedded_decoder, hidden)
        elif self.cell_type == 'rnn':
            output, hidden = self.rnn(embedded_decoder, hidden)

        output = F.relu(self.out(output))  # Apply ReLU activation to the output
        output = F.log_softmax(output, dim=-1)  # Apply log softmax activation to obtain output probabilities

        return output, hidden, cell  # Return the output, hidden state, and cell state

    def init_hidden(self, encoder_hidden, encoder_cell, encoder_bidirectional):
        hidden = encoder_hidden[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the hidden state using the encoder's hidden state
        cell = encoder_cell[-(1 + int(encoder_bidirectional)): ].repeat(self.num_layers, 1, 1)  # Initialize the cell state using the encoder's cell state
        return hidden, cell  # Return the initialized hidden state and cell state



class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder  # Initialize the encoder
        self.decoder = decoder  # Initialize the decoder
        self.device = device  # Store the device (e.g., CPU or GPU)
        self.max_target_length = 0  # Variable to store the maximum target sequence length
        self.sos = 0  # Start-of-sequence token
        
    def forward(self, source, target, teacher_forcing_ratio=0.5):
        batch_size = target.shape[0]  # Get the batch size
        target_len = target.shape[1]  # Get the target sequence length
        self.max_target_length = target_len  # Update the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor
        
        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state
        
        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU
        
        input = target[:, 0]  # Set the first input to the decoder as the <sos> token
        self.sos = target[:, 0]  # Store the <sos> token
        
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        # Initialize the decoder hidden state and cell state
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder.forward(input, hidden, cell)
            # Forward pass through the decoder
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            teacher_force = random.random() < teacher_forcing_ratio  # Determine whether to use teacher forcing
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = target[:, t] if teacher_force else top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs
    
    def inference(self, source, target):
        batch_size = source.shape[0]  # Get the batch size
        target_len = self.max_target_length  # Get the maximum target sequence length
        target_vocab_size = self.decoder.output_size  # Get the target vocabulary size
        
        outputs = torch.zeros(batch_size, target_len, target_vocab_size).to(self.device)  # Initialize the outputs tensor
        
        encoder_hidden, encoder_cell = self.encoder.init_hidden(batch_size)  # Initialize the encoder hidden state and cell state
        
        if (self.encoder.cell_type == 'lstm'):
            encoder_outputs, encoder_hidden, encoder_cell = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder LSTM
        if (self.encoder.cell_type == 'rnn'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder RNN
        if (self.encoder.cell_type == 'gru'):
            encoder_outputs, encoder_hidden = self.encoder.forward(source, encoder_hidden, encoder_cell)
            # Forward pass through the encoder GRU
        
        input = self.sos  # Set the first input to the <sos> token
        
        hidden, cell = self.decoder.init_hidden(encoder_hidden, encoder_cell, self.encoder.bidirectional)
        # Initialize the decoder hidden state and cell state
        
        for t in range(1, target_len):
            output, hidden, cell = self.decoder.forward(input, hidden, cell)
            # Forward pass through the decoder
            outputs[:, t] = output.squeeze(1)  # Store the decoder output in the outputs tensor
            top1 = output.argmax(-1)  # Get the index of the highest probability output
            input = top1.squeeze(1)  # Set the next input to the decoder
            
        return outputs





In [ ]:
def create_model(input_dict, target_dict):
    variables = {}
    
    # Count the number of elements in the input and target dictionaries
    variables['a'] = len(input_dict)
    variables['b'] = len(target_dict)
    
    # Set values for variables c, d, e, f, g, h, i, j, k, and max_length
    variables['c'] = 32    #batch size
    variables['d'] = 32     #val batch size
    variables['e'] = 256    #enc_embedding
    variables['f'] = 256    #dec embedding
    variables['g'] = 512    #hidden_size 
    variables['h'] = 3     #enc_number_layers
    variables['i'] = 2     #dec_num_layers
    variables['j'] = 0.4    #enc_dropout
    variables['k'] = 0.4    #dec_drop_out
    variables['max_length'] = max_target_length
    
    # Set value for variable l
    variables['l'] = 'gru'
    
    # Create an instance of EncoderRNN and assign it to variable m
    variables['m'] = EncoderRNN(device, variables['l'], variables['a'], variables['e'], variables['g'],
                                variables['h'], bidirectional=True, dropout_p=variables['j'])
    
    # Create an instance of DecoderRNN and assign it to variable n
    variables['n'] = DecoderRNN(device, variables['l'], variables['b'], variables['f'], variables['g'],
                                variables['max_length'], variables['k'], variables['i'], bidirectional=True)
    
    # Create an instance of Seq2Seq by passing in the encoder and decoder instances, and assign it to variable o
    variables['o'] = Seq2Seq(variables['m'], variables['n'], device).to(device)
    
    # Define a function count_parameters(model) that returns the total number of trainable parameters in a model
    def count_parameters(model):
        return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
    # Print the number of trainable parameters in the Seq2Seq model
    print(f' {count_parameters(variables["o"]):,} To be train params')
    
    # Create an instance of Adam optimizer and assign it to variable p
    variables['p'] = torch.optim.Adam(variables['o'].parameters(), lr=0.001)
    
    # Create an instance of negative log likelihood loss and assign it to variable q
    variables['q'] = nn.NLLLoss()
    
    # Return the Seq2Seq model, optimizer, and criterion
    return variables['o'], variables['p'], variables['q']

# Call the create_model function with input_dict and target_dict, and assign the returned values to model, optimizer, and criterion respectively
model, optimizer, criterion = create_model(input_dict, target_dict)


In [ ]:
def trainIters(m, p, b, n, o, t, p_e=10, p_v=10, l=True, A=False):
    s = time.time()  # Starting time

    p_l = []  # List to store training loss
    t_w_a = []  # List to store word-level accuracy on training set
    v_w_a = []  # List to store word-level accuracy on validation set
    p_l_t = 0  # Training loss per iteration
    p_l_v_t = 0  # Validation loss per iteration

    t_p = p[0]  # Training data
    v_p = p[1]  # Validation data
    t_a = 0  # Accumulated word-level accuracy on training set

    c = nn.NLLLoss()  # Negative log likelihood loss

    count = 0  # Iteration count
    for it in range(1, n + 1):  # Number of iterations
        for i in range(0, len(t_p) - b, b):  # Batch processing
            t_a = 0  # Reset accumulated word-level accuracy
            count += 1  # Increment iteration count

            if i + b > len(t_p):
                b = len(t_p) - i + 1  # Adjust batch size if remaining samples are less than the batch size

            i_t = torch.stack([t_p[i + j][0] for j in range(b)]).squeeze(1).long().cuda()  # Input tensor
            t_t = torch.stack([t_p[i + j][1] for j in range(b)]).squeeze(1).long().cuda()  # Target tensor

            o.zero_grad()  # Clear gradients
            out = m(i_t, t_t, teacher_forcing_ratio=t if count < 4000 else 0)  # Forward pass
            out = torch.permute(out, [0, 2, 1])  # Permute output tensor dimensions
            l = c(out, t_t)  # Calculate loss

            t_a_w = word_level_accuracy(t_t, out) * b  # Calculate word-level accuracy on training set
            t_a += t_a_w  # Accumulate word-level accuracy
            l.backward()  # Backward pass
            torch.nn.utils.clip_grad_norm_(m.parameters(), 1)  # Clip gradients to avoid exploding gradients
            o.step()  # Update model parameters

            p_l_t += l  # Accumulate training loss per iteration
            p_l_v_t += l  # Accumulate validation loss per iteration

            if count % 800 == 0:
                v_i_t = torch.stack([v_p[j][0] for j in range(b)]).squeeze(1).long().cuda()  # Validation input tensor
                v_t_t = torch.stack([v_p[j][1] for j in range(b)]).squeeze(1).long().cuda()  # Validation target tensor

                if A:
                    v_o, _ = m.inference(v_i_t, v_t_t)
                else:
                    v_o = m.inference(v_i_t, v_t_t)

                v_o = v_o.permute(0, 2, 1)  # Permute validation output tensor dimensions
                v_l = c(v_o, v_t_t)  # Calculate validation loss
                v_l_s = v_l
                wandb.log({'Val_Loss': v_l_s})  # Log validation loss

            if count % 800 == 0:
                p_l_a = p_l_t / 800  # Average training loss over 800 iterations
                p_l_t = 0  # Reset training loss accumulator
                print('%s (%d %d%%) %.7f' % (timeSince(s, it / n), it, it / n * 100, p_l_a))  # Print progress

            if count % 800 == 0:
                p_l_a = p_l_v_t / 800  # Average validation loss over 800 iterations
                p_l.append(p_l_a.detach())  # Append training loss to list
                wandb.log({'Train Loss': p_l_a})  # Log training loss
                p_l_v_t = 0  # Reset validation loss accumulator

        t_a = t_a / (len(t_p) - b)  # Calculate average word-level accuracy on training set
        t_w_a.append(t_a)  # Append word-level accuracy to list

    print(t_w_a)  # Print word-level accuracy on training set

    p_l = [l.cpu().numpy() for l in p_l]  # Convert training loss to numpy array
    p_l_s = sample_equidistant_points(p_l, n)  # Sample equidistant points from the training loss

    w_c = 0  # Total correct predictions
    for i in range(0, len(v_p) - b, b):  # Batch processing on validation set
        if i + b > len(v_p):
            b = len(v_p) - i + 1  # Adjust batch size if remaining samples are less than the batch size

        v_i_t = torch.stack([v_p[i + j][0] for j in range(b)]).squeeze(1).long().cuda()  # Validation input tensor
        v_t_t = torch.stack([v_p[i + j][1] for j in range(b)]).squeeze(1).long().cuda()  # Validation target tensor

        if A:
            v_o, _ = m.inference(v_i_t, v_t_t)
        else:
            v_o = m.inference(v_i_t, v_t_t)

        v_o = v_o.permute(0, 2, 1)  # Permute validation output tensor dimensions
        v_l = c(v_o, v_t_t)  # Calculate validation loss

        v_a_w = word_level_accuracy(v_t_t, v_o)  # Calculate word-level accuracy on validation set
        w_c += v_a_w * b  # Accumulate correct predictions

    w_a = w_c / (len(v_p) - b)  # Average word-level accuracy on validation set
    m = {'Val_Accuracy': w_a}  # Log validation accuracy
    wandb.log(m)

    print(f"Val loss = {v_l}")  # Print validation loss
    print(f'Word-level-accuracy on val set = {w_a}')  # Print word-level accuracy on validation set


In [ ]:
# Defining the sweep configuration
sw = {
    'method': 'bayes',
    'name': 'Hyperparameter Tuning-Bayesian'
}

# Defining the metric configuration
m = {
    'name': 'Val_Accuracy',
    'goal': 'maximize'
}

# Adding the metric configuration to the sweep configuration
sw['metric'] = m

# Defining the parameters configuration
p = {
    'optimiser': {'values': ['adam','nadam']},
    'teacher_forcing_ratio': {'values': [0.3, 0.5, 0.7]},
    'bidirectional': {'values': [True, False]},
    'enc_embedding': {'values': [128,256]},
    'dec_embedding': {'values': [128,256]},
    'epochs': {'values': [2,5,8]},
    'hidden_size': {'values': [64,128,256,512]},
    'enc_layers': {'values': [2,3]},
    'dec_layers': {'values': [2,3]},
    'dropout': {'values': [0.3,0.4]},
    'cell_type': {'values': ['lstm','gru','rnn']}
}

# Adding the parameters configuration to the sweep configuration
sw['parameters'] = p

# Creating a sweep using the sweep configuration and assigning it to sw_id
sw_id = wandb.sweep(sw, project="CS6910 Assignment 3")

# Function to train the sweep
def train_sweep(c=None):
    # Initializing wandb run with the given config
    with wandb.init(config=c) as r:
        c = wandb.config

        # Creating the validation data dictionary
        vd = {
            'input_dim': len(input_dict),
            'output_dim': len(target_dict),
            'batch_size': 32,
            'val_batch_size': 32,
            'enc_embedding': c.enc_embedding,
            'dec_embedding': c.dec_embedding,
            'hidden': c.hidden_size,
            'enc_num_layers': c.enc_layers,
            'dec_num_layers': c.dec_layers,
            'enc_dropout': c.dropout,
            'dec_dropout': c.dropout,
            'max_length': max_target_length,
            'cell_type': c.cell_type
        }

        # Extracting values from the validation data dictionary
        input_dim = vd['input_dim']
        output_dim = vd['output_dim']
        batch_size = vd['batch_size']
        val_batch_size = vd['val_batch_size']
        enc_embedding = vd['enc_embedding']
        dec_embedding = vd['dec_embedding']
        hidden = vd['hidden']
        enc_num_layers = vd['enc_num_layers']
        dec_num_layers = vd['dec_num_layers']
        enc_dropout = vd['enc_dropout']
        dec_dropout = vd['dec_dropout']
        max_length = vd['max_length']
        cell_type = vd['cell_type']

        # Creating the encoder and decoder models
        enc = EncoderRNN(device, cell_type, input_dim, enc_embedding, hidden, enc_num_layers,
                         bidirectional=c.bidirectional, dropout_p=enc_dropout)
        dec = DecoderRNN(device, cell_type, output_dim, dec_embedding, hidden, max_length, dec_dropout,
                         dec_num_layers, bidirectional=c.bidirectional)
        model = Seq2Seq(enc, dec, device).to(device)

        # Setting the experiment name based on the config
        exp_name = f"{c.cell_type}_e_{c.optimiser}"

        # Assigning the experiment name to the wandb run
        wandb.run.name = exp_name

        # Creating the optimizer based on the chosen optimiser
        optimizer = torch.optim.Adam(model.parameters(), lr=0.001) if c.optimiser == 'adam' else torch.optim.NAdam(
            model.parameters(), lr=0.001)

        criterion = nn.NLLLoss()


        # Training the model using the trainIters function
        trainIters(model, pairs, 32, c.epochs, optimizer, c.teacher_forcing_ratio)

# Running the sweep agent with the given sweep ID and train_sweep function
wandb.agent(sw_id, train_sweep, count=1)

# Finishing the wandb run
wandb.finish()

In [ ]:
def BestVanillaModel():
    variables = {
        'a': len(input_dict),  # Input dimension
        'b': len(target_dict),  # Output dimension
        'c': 32,  # Batch_size
        'd': 32,  # val_batch_size
        'e': 128,  # enc_embedding
        'f': 128,  # dec_embedding
        'g': 512,  # hidden
        'h': 10,  # epochs
        'i': 3,  # enc_num_layers
        'j': 3,  # dec_num_layers
        'k': 0.3,  # enc_dropout
        'l': 0.3,  # dec_dropout
        'm': max_target_length,
        'n': 'lstm',
        'o': 0.7  # teacher_forcing_ratio
    }

    p = EncoderRNN(device, variables['n'], variables['a'], variables['e'], variables['g'], variables['i'], bidirectional=True, dropout_p=variables['k'])
    q = DecoderRNN(device, variables['n'], variables['b'], variables['f'], variables['g'], variables['m'], variables['l'], variables['j'], bidirectional=True)
    r = Seq2Seq(p, q, device).to(device)

    s = torch.optim.Adam(r.parameters(), lr=0.001)

    wandb.init()

    trainIters(r, pairs, variables['c'], variables['h'], s, variables['o'])

BestVanillaModel()

In [ ]:
# Set the values of variables using dictionaries
variables = {
    'a': len(input_dict),
    'b': len(target_dict),
    'c': 32,  # Batch_size
    'd': 32,  #val_batch_size
    'e': 128, #enc_embedding
    'f': 128,#dec_embedding
    'g': 256,#hidden
    'h': 10,#epochs
    'i': 2,#enc_num_layers
    'j': 2,#dec_num_layers
    'k': 0.3,#enc_dropout
    'l': 0.3,#dec_dropout
    'm': max_target_length,
    'n': 'lstm',
    'o': 0.7#teacher_forcing_ratio
}

# Create the encoder, decoder, and Seq2Seq models
p = EncoderRNN(device, variables['n'], variables['a'], variables['e'], variables['g'], variables['i'], bidirectional=True, dropout_p=variables['k'])
q = DecoderRNN(device, variables['n'], variables['b'], variables['f'], variables['g'], variables['m'], variables['l'], variables['j'], bidirectional=True)
r = Seq2Seq(p, q, device).to(device)

# Define the optimizer
s = torch.optim.Adam(r.parameters(), lr=0.001)

# Initialize wandb
wandb.init()

# Train the model using trainIters function
trainIters(r, pairs, variables['c'], variables['h'], s, variables['o'])


import wandb
import csv
import torch
import numpy as np

wandb.init(project='Ass-3-test-accuracy-plot')

class MyClass:
    @staticmethod
    def append_strings_to_csv(file_path, s1, s2, s3):
        with open(file_path, 'a', newline='') as csvfile:
            writer = csv.writer(csvfile)
            writer.writerow([s1, s2, s3])

    def train_and_evaluate(self):
        # Prepare for testing
        t = pairs[2]
        u = 0
        v = 1
        w = 'VanillaFinalPredictions' + str(v)
        x = '/content/drive/MyDrive/' + w + '.csv'

        # Iterate over test_pairs in batches
        for y in np.arange(start=0, stop=len(t) - variables['c'], step=variables['c']):
            if y + variables['c'] > len(t):
                variables['c'] = len(t) - y + 1
            z = []
            aa = []

            # Gather input and target tensors for the batch
            for ab in range(variables['c']):
                z.append(t[y + ab][0])
                aa.append(t[y + ab][1])

            z = torch.stack(z).squeeze(1).long().cuda()
            aa = torch.stack(aa).squeeze(1).long().cuda()

            # Perform inference and calculate the loss
            ac = r.inference(z, aa)
            ac = ac.permute(0, 2, 1)
            ad = criterion(ac, aa)

            # Iterate over each example in the batch
            for ae in range(variables['c']):
                af = print_keys_for_values(input_dict, z[ae])
                ag = print_keys_for_values(target_dict, torch.argmax(ac[ae], dim=0))
                ah = print_keys_for_values(target_dict, aa[ae])
                self.append_strings_to_csv(x, af, ah, ag)

            # Calculate word-level accuracy
            ai = word_level_accuracy(aa, ac)
            u = u + (ai) * variables['c']

        # Calculate and log the accuracy on the test set
        aj = u / (len(t) - variables['c'])
        wandb.log({'Test Accuracy': aj})

        print(f"Accuracy on Test Set is {aj}")

wandb.init(project='Ass-3-test-accuracy-plot')

# Create an instance of MyClass
my_object = MyClass()

# Call the train_and_evaluate method
my_object.train_and_evaluate()